In [8]:
from chop.models import ModelFactory

print(ModelFactory._model_info_dict.keys())  # 查看所有已注册的模型


dict_keys(['bert-base-uncased', 'cnv-toy', 'cnv', 'cnv_residual', 'cswin_64_12211_tiny_224', 'cswin_64_24322_small_224', 'cswin_96_24322_base_224', 'cswin_144_24322_large_224', 'cswin_96_24322_base_384', 'cswin_144_24322_large_384', 'deit_tiny_patch16_224', 'deit_small_patch16_224', 'deit_base_patch16_224', 'deit_tiny_distilled_patch16_224', 'deit_small_distilled_patch16_224', 'deit_base_distilled_patch16_224', 'deit_base_patch16_384', 'deit_base_distilled_patch16_384', 'efficientnet_b0', 'efficientnet_b3', 'efficientnet_v2_s', 'efficientnet_v2_m', 'efficientnet_v2_l', 'jsc-toy', 'jsc-tiny', 'jsc-s', 'jsc-trt', 'lfc', 'mobilenetv2', 'mobilenetv3_small', 'mobilenetv3_large', 'nerf', 'opt-patched', 'pvt_tiny', 'pvt_small', 'pvt_medium', 'pvt_large', 'pvt_v2', 'repvgg_a0', 'repvgg_a1', 'repvgg_a2', 'repvgg_b0', 'repvgg_b1', 'repvgg_b1g2', 'repvgg_b1g4', 'repvgg_b2', 'repvgg_b2g2', 'repvgg_b2g4', 'repvgg_b3', 'repvgg_b3g2', 'repvgg_b3g4', 'repvgg_d2se', 'resnet18', 'resnet34', 'resnet50', 

In [13]:
from transformers import AutoModelForSequenceClassification

checkpoint = "prajjwal1/bert-tiny"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

# 保存权重
save_path = "./bert_tiny.ckpt"
model.save_pretrained(save_path)  # 这会保存 `pytorch_model.bin` 和 `config.json`
print(f"模型已保存到 {save_path}")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


模型已保存到 ./bert_tiny.ckpt


model.safetensors:   0%|          | 0.00/17.7M [00:00<?, ?B/s]

In [51]:
import sys
import logging
import os
from pathlib import Path
from pprint import pprint as pp

import torch
import torchvision
import torchvision.transforms as transforms
import torch_tensorrt

machop_path = Path(".").resolve().parent/"machop"
assert machop_path.exists(), "Failed to find machop at: {}".format(machop_path)
sys.path.append(str(machop_path))

from chop.dataset import MaseDataModule, get_dataset_info
from chop.tools.logger import set_logging_verbosity

import chop.passes as passes
from chop.tools.get_input import InputGenerator
from chop.tools.checkpoint_load import load_model
from chop.ir import MaseGraph
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from chop.models import get_model_info, get_model

set_logging_verbosity("info")

# ------------------
# 1. 加载模型与 tokenizer
# ------------------
model_name = "prajjwal1/bert-tiny"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
model.eval()

# 如果 Torch-TensorRT 或当前环境对 scaled_dot_product_attention 兼容性不好，可以尝试禁用：
# model.config.enable_scaled_dot_product_attention = False

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# ------------------
# 2. 准备数据
# ------------------
dataset_name = "sst2"
batch_size = 8  # 示例

data_module = MaseDataModule(
    name=dataset_name,
    batch_size=batch_size,
    model_name=model_name,
    num_workers=0,
)
data_module.prepare_data()
data_module.setup()


# a demonstration of how to feed an input value to the model
dummy_input = tokenizer(
    [
        "AI may take over the world one day",
        "This is why you should learn ADLS",
    ],
    return_tensors="pt",
)

# generate the mase graph and initialize node metadata
mg = MaseGraph(model)
# 初始化元数据
mg, _ = passes.init_metadata_analysis_pass(mg)

# 添加通用元数据（包含操作类型推理和形状传播）
mg, _ = passes.add_common_metadata_analysis_pass(
    mg,
    pass_args={
        "dummy_in": dummy_input,
        "add_value": False,
    },
)

INFO     Set logging level to info
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
`past_key_values` were not specified as input names, but model.config.use_cache = True. Setting model.config.use_cache = False.


tensor([[ 101, 9932, 2089, 2202, 2058, 1996, 2088, 2028, 2154,  102],
        [ 101, 2023, 2003, 2339, 2017, 2323, 4553, 4748, 4877,  102]])
tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
tensor([[ 101, 9932, 2089, 2202, 2058, 1996, 2088, 2028, 2154,  102],
        [ 101, 2023, 2003, 2339, 2017, 2323, 4553, 4748, 4877,  102]])
tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])
tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])
tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])
tensor([[[[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]]],


        [[[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]]]])
tensor([[[[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
   

In [38]:
# 检查输入形状
print(f"Input shape: {dummy_input['input_ids'].size()}")

Input shape: torch.Size([2, 10])


In [ ]:
from torch_tensorrt.ts import ptq

# 使用 torch_tensorrt 进行量化转换
testing_dataloader = data_module.test_dataloader()

calibrator = ptq.DataLoaderCalibrator(
    testing_dataloader,
    cache_file="./calibration.cache",
    use_cache=False,
    algo_type=torch_tensorrt.ptq.CalibrationAlgo.MINMAX_CALIBRATION,  # NLP任务适合MinMax校准
    device=torch.device("cuda:0"),
)

# 定义 TensorRT 输入格式（NLP任务）
inputs = [
    torch_tensorrt.Input(
        min_shape=[1, 16],   # 最小 batch_size=1, 序列长度=16
        opt_shape=[8, 32],   # 最优 batch_size=8, 序列长度=32
        max_shape=[16, 128], # 最大 batch_size=16, 序列长度=128
        dtype=torch.int32,    # NLP 任务一般是 int32 或 int64
    )
]

trt_mod = torch_tensorrt.compile(model, inputs=[torch_tensorrt.Input((8, 3, 32, 32))],
                                    enabled_precisions={torch.float, torch.half, torch.int8},
                                    calibrator=calibrator,
                                    device={
                                         "device_type": torch_tensorrt.DeviceType.GPU,
                                         "gpu_id": 0,
                                         "dla_core": 0,
                                         "allow_gpu_fallback": False,
                                         "disable_tf32": False
                                     })

RuntimeError: The test dataset is not availableprobably because the test set does not have ground truth labels, or the test dataset does not exist. For the former case, try predict_dataloader

In [49]:
# 使用动态形状编译并保存 TorchScript 模块

inputs = [
    torch_tensorrt.Input(
        min_shape=[1, 16],
        opt_shape=[50, 16],
        max_shape=[100, 16],
        dtype=torch.half,
    )
]
enabled_precisions = {torch.float, torch.half}  # Run with fp16

trt_ts_module = torch_tensorrt.compile(
    model, inputs=inputs, enabled_precisions=enabled_precisions
)

input_data = torch.randn(100, 16).to('cuda')
input_data = input_data.to("cuda").half()
result = trt_ts_module(input_data)
torch.jit.save(trt_ts_module, "trt_ts_module.ts")

E0303 19:56:01.955000 60828 site-packages/torch/export/_trace.py:1021] always_classified is unsupported.
E0303 19:56:01.956000 60828 site-packages/torch/export/_trace.py:1021] always_classified is unsupported.


TorchRuntimeError: Failed running call_module L__self___bert_embeddings_word_embeddings(*(FakeTensor(..., device='cuda:0', size=(s0, 16), dtype=torch.float16),), **{}):
tensors used as indices must be long, int, byte or bool tensors

from user code:
   File "/srcPkgs/miniconda3/lib/python3.11/site-packages/transformers/models/bert/modeling_bert.py", line 1665, in forward
    outputs = self.bert(
  File "/srcPkgs/miniconda3/lib/python3.11/site-packages/torch/nn/modules/module.py", line 1750, in _call_impl
    return forward_call(*args, **kwargs)
  File "/srcPkgs/miniconda3/lib/python3.11/site-packages/transformers/models/bert/modeling_bert.py", line 1078, in forward
    embedding_output = self.embeddings(
  File "/srcPkgs/miniconda3/lib/python3.11/site-packages/torch/nn/modules/module.py", line 1750, in _call_impl
    return forward_call(*args, **kwargs)
  File "/srcPkgs/miniconda3/lib/python3.11/site-packages/transformers/models/bert/modeling_bert.py", line 211, in forward
    inputs_e

In [ ]:
# 将原模型转换为半精度，并用转换后的模型进行一次前向推理
half_model = model.half()
half_model(input_data)

In [ ]:
import time
# 测量推理时间
inputs = torch.randn(8, 3, 32, 32).to('cuda')
timings = []

start_time = time.time()
features = model.to('cuda')(inputs)
torch.cuda.synchronize()
end_time = time.time()
timings.append(end_time - start_time)


start_time = time.time()
features = trt_mod(inputs)
torch.cuda.synchronize()
end_time = time.time()
timings.append(end_time - start_time)

timings


[0.002302885055541992, 0.0005021095275878906]

In [ ]:
mg.model = trt_mod # 存储的模型替换为 TensorRT 优化模型

In [ ]:
mg.fx_graph #显示转换后的 torch.fx 图

AttributeError: 'MaseGraph' object has no attribute 'onnx_model'

In [19]:
mg.fx_graph

'torch.fx.graph'